In [21]:
checkpoint="checkpoint-100"
model_checkpoint="/root/zindi/models/marian/marian_output/"+checkpoint
model_checkpoint_ct="/root/zindi/models/marian/ct/"+checkpoint

In [22]:
import tqdm as notebook_tqdm

In [12]:
### Eval original model 
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
from sacrebleu.metrics import BLEU
from tqdm import tqdm

def translate(model, tokenizer, text, device):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=150).to(device)
    translated = model.generate(**inputs)
    return tokenizer.batch_decode(translated, skip_special_tokens=True)

def calculate_bleu(references, hypotheses):
    bleu = BLEU()
    return bleu.corpus_score(hypotheses, [references]).score

def validate_model(model_name, dataset_name):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Load model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

    # Load dataset
    dataset = load_dataset(dataset_name, split="validation")

    # Translate and calculate BLEU score
    hypotheses = []
    references = []

    for batch in tqdm(dataset, desc="Translating"):
        source_text = batch['translation']['dyu']  # Adjust this based on your dataset's column names
        reference = batch['translation']['fr']  # Adjust this based on your dataset's column names
        
        translation = translate(model, tokenizer, source_text, device)[0]
        # print(reference)
        # print(translation)
        
        hypotheses.append(translation)
        references.append(reference)
        # break

    bleu_score = calculate_bleu(references, hypotheses)
    print(f"BLEU Score: {bleu_score:.2f}")

# Run validation with GPU
# validate_model(model_checkpoint, "uvci/Koumankan_mt_dyu_fr", use_gpu=True)

# Run validation without GPU
validate_model(model_checkpoint, "uvci/Koumankan_mt_dyu_fr")

Using device: cuda


Translating: 100%|██████████| 1471/1471 [02:23<00:00, 10.24it/s]

BLEU Score: 6.21


In [23]:
### Eval ct model
# import torch
# from transformers import AutoTokenizer
import sentencepiece as spm
import ctranslate2
from datasets import load_dataset
from sacrebleu.metrics import BLEU
from tqdm import tqdm


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

sp_source_model = spm.SentencePieceProcessor(model_file=model_checkpoint_ct+'/source.spm')
sp_target_model = spm.SentencePieceProcessor(model_file=model_checkpoint_ct+'/target.spm')


def translate(translator, text):
    tokens = sp_source_model.encode(text, out_type=str)
    # print(tokens)
    tokens = ['dyu'] + tokens + ["</s>"] +['fr']
    try:
        results = translator.translate_batch([tokens])
        # The translated results are token strings, so we need to convert them to IDs before decoding
        translations = []
        for translation in results:
            decoded_text = sp_target_model.decode(translation.hypotheses[0])
            translations.append(decoded_text)
    except Exception as e:
        print(f"Translation error: {e}")
        translations = [""]  # Return empty string if translation fails
    return translations

def calculate_bleu(references, hypotheses):
    bleu = BLEU()
    return bleu.corpus_score(hypotheses, [references]).score

def validate_model(model_path, dataset_name):
    # Load CTranslate2 model
    translator = ctranslate2.Translator(model_path, device=device)
    # Load dataset
    dataset = load_dataset(dataset_name, split="validation")
    # Translate and calculate BLEU score
    hypotheses = []
    references = []

    for batch in tqdm(dataset, desc="Translating"):
        source_text = batch['translation']['dyu']  # Adjust this based on your dataset's column names
        reference = batch['translation']['fr']  # Adjust this based on your dataset's column names
        
        # translation = translate(translator, tokenizer, source_text, device)[0]
        # tokenizer=None
        translation = translate(translator, source_text)[0]
        hypotheses.append(translation)
        references.append(reference)
        # print(hypotheses,"----", reference)
        # print(translation, "----", reference)
        # break

    bleu_score = calculate_bleu(references, hypotheses)
    print(f"BLEU Score: {bleu_score:.2f}")

validate_model(model_checkpoint_ct, "uvci/Koumankan_mt_dyu_fr")

Using device: cuda


Translating: 100%|██████████| 1471/1471 [00:34<00:00, 42.91it/s]


BLEU Score: 4.78
